# 주어진 Crop 영역을 새로운 이미지 파일로 만들기 위한 작업

## data prepration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm

import os
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
def load_image_path(dir):
    food_middle_list = sorted(os.listdir(dir))

    food_list = [] # 음식이름
    food_path_list = [] # 음식이름 경로

    for food_middle in food_middle_list:
        middle_path = os.path.join(dir,food_middle)
        food_name_list = os.listdir(middle_path)

        for food in food_name_list:
            path = os.path.join(middle_path,food)
            food_path_list.append(path)

            name = path.split('/')[-1]
            food_list.append(name)

    return food_list, food_path_list


In [ ]:
data_dir = '/content/drive/MyDrive/project3/data/traindata/kfood'
food_list, food_path_list = load_image_path(data_dir)

print(len(food_list))
print(food_list)
print(food_path_list)

99
['갈비구이', '갈치구이', '고등어구이', '곱창구이', '닭갈비', '장어구이', '조개구이', '떡갈비', '불고기', '삼겹살', '더덕구이', '조기구이', '황태구이', '훈제오리', '콩나물국', '계란국', '떡국_만두국', '시래기국', '무국', '북엇국', '미역국', '육개장', '과메기', '편육', '콩자반', '양념치킨', '피자', '후라이드치킨', '젓갈', '갓김치', '깍두기', '나박김치', '무생채', '배추김치', '부추김치', '열무김치', '백김치', '총각김치', '오이소박이', '파김치', '가지볶음', '고사리나물', '미역줄기볶음', '숙주나물', '시금치나물', '애호박볶음', '송편', '꿀떡', '경단', '만두', '장', '적', '장아찌', '전', '전골', '조림', '라면', '쫄면', '짬뽕', '물냉면', '짜장면', '비빔냉면', '수제비', '막국수', '잔치국수', '열무국수', '칼국수', '콩국수', '도토리묵', '콩나물무침', '꽈리고추무침', '도라지무침', '잡채', '홍어무침', '회무침', '김밥', '김치볶음밥', '누룽지', '알밥', '비빔밥', '자

## Create Boxed Images

In [ ]:
def make_new_image(raw_image_path):
    crop_file_name = 'crop_area.properties'
    crop_file_path = os.path.join(raw_image_path, crop_file_name)

    boxed_image_path = raw_image_path.replace('kfood', 'boxed_image')
    crop_image_path = raw_image_path.replace('kfood', 'crop_image')

    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(boxed_image_path):
        os.makedirs(boxed_image_path)
    if not os.path.exists(crop_image_path):
        os.makedirs(crop_image_path)

    # 딕셔너리 초기화
    crop_areas = {}
    error_list = []

    print(raw_image_path)
    # 파일 읽기
    with open(crop_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

        for line in tqdm(lines, desc="Processing images"):

            # 각 줄에서 = 을 기준으로 키-값 쌍 분리
            key, value = line.strip().split('=')
            # 값을 정수 리스트로 변환
            # 좌표값이 틀리게 저장된 경우 존재
            try:
                coordinates = list(map(int, value.split(',')))
            except:
                error_list.append(f"좌표값 오류 {key}: {value}")
                continue
            # 딕셔너리에 저장
            crop_areas[key] = coordinates

            # 이미지 파일 경로 - jpg, png, bmp 확장자 확인
            img_extensions = ['.jpg', '.jpeg', '.JPG', '.JEPG', '.png', '.PNG', '.bmp', '.BMP']
            image_path = None

            for ext in img_extensions:
                potential_path = os.path.join(raw_image_path, f"{key}{ext}")
                if os.path.exists(potential_path):
                    image_path = potential_path
                    break

            # 이미지 읽기
            if image_path is not None and os.path.isfile(image_path):
                # RGBA 형식의 파일들이 존재 - 알파 채널 포함하여 읽기
                image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

                if image is not None:
                    x, y, w, h = coordinates

                    # Save Cropped image
                    ## Crop Box
                    crop_img = image[y:y+h, x:x+w]
                    output_crop_image_path = os.path.join(crop_image_path, f"crop_{key}.jpg")
                    ## Save
                    cv2.imwrite(output_crop_image_path, crop_img)

                    # Save Boxed image
                    ## Draw Box
                    boxed_image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    ## save
                    box_image_path = os.path.join(boxed_image_path, f"boxed_{key}.jpg")
                    cv2.imwrite(box_image_path, boxed_image)
                else :
                    error_list.append(f"Image can't read: {key}")
            else :
                error_list.append(f"Image not found: {key}")

        # error list 출력
        print('\n')
        for error in error_list:
            print(error)
        print('------------------------')


In [ ]:
for food_path in food_path_list:
    make_new_image(food_path)

In [ ]:
raw_image_path = food_path_list[2]
make_new_image(raw_image_path)

/content/drive/MyDrive/project3/data/traindata/kfood/구이/고등어구이


Processing images: 100%|██████████| 219/219 [00:10<00:00, 20.12it/s]



좌표값 오류 Img_002_0645: ,852,535
좌표값 오류 Img_002_0894: ,1433,824
좌표값 오류 Img_002_0898: ,854,503
좌표값 오류 Img_002_0917: ,1934,1077
------------------------
